In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
import platform

In [2]:
# Load environment variables
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Initialize clients
openai = OpenAI()
claude = anthropic.Anthropic()

# Model settings
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
processor = platform.machine()

# OPENAI_MODEL = "gpt-4o"
# CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
# Language definitions
programming_languages = [
    {"name": "C++", "extension": "cpp", "fn": None},
    {"name": "Javascript", "extension": "js", "fn": None},
    {"name": "Php", "extension": "php", "fn": None}
]


In [5]:
# Prompt generation
def get_name_by_extension(extension):
    for lang in programming_languages:
        if lang["extension"] == extension:
            return lang["name"]
    return None

In [6]:

def get_system_message(prog_lang):
    name = get_name_by_extension(prog_lang)
    return (
        f"You are an assistant that reimplements Python code to {name} for an {processor} device. "
        f"Respond only with code; use comments sparingly and do not provide any explanation other than occasional comments. "
        f"The {name} response needs to produce an identical output in the fastest possible time. "
        f"If the used function does not exist for {name}, interchange it for its compatibility and if not, throw an error."
    )

In [7]:
def user_prompt_for(python, prog_lang):
    name = get_name_by_extension(prog_lang)
    return (
        f"Rewrite this Python code in {name} with the fastest possible implementation that produces identical output in the least time. "
        f"Respond only with {name} code; do not explain your work other than a few comments. "
        f"Pay attention to number types to ensure no int overflows\n\n{python}"
    )

In [8]:
def messages_for(python, prog_lang):
    return [
        {"role": "system", "content": get_system_message(prog_lang)},
        {"role": "user", "content": user_prompt_for(python, prog_lang)}
    ]

In [9]:
# Streaming responses
def stream_gpt(python, prog_lang):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(python, prog_lang),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n', '').replace('javascript\n', '').replace('```', '')

In [10]:
# Sample Python code
python_hard = """# Be careful to support large number sizes
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000
initial_seed = 42
min_val = -10
max_val = 10

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [11]:
def stream_claude(python, prog_lang):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=get_system_message(prog_lang),
        messages=[{"role": "user", "content": user_prompt_for(python, prog_lang)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n', '').replace('javascript\n', '').replace('```', '')

In [12]:
def optimize(python, model, prog_lang):
    if model == "GPT":
        result = stream_gpt(python, prog_lang)
    elif model == "Claude":
        result = stream_claude(python, prog_lang)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [13]:
# Gradio UI
with gr.Blocks() as ui:
    gr.Markdown("# 🔁 Convert Python Code to Other Languages")
    with gr.Row():
        with gr.Column():
            python = gr.Textbox(label="Python code:", value=python_hard, lines=20)

    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")

    for lang in programming_languages:
        with gr.Row():
            convert = gr.Button(f"Convert to {lang['name']}")
            converted_code = gr.Textbox(label=f"{lang['name']} Code", lines=20)
            current_selected = gr.Dropdown([lang["extension"]], value=lang["extension"], visible=False)
            convert.click(optimize, inputs=[python, model, current_selected], outputs=[converted_code])

# Launch UI
if __name__ == "__main__":
    ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
